In [32]:
import pandas as pd

df = pd.DataFrame(
    {
        "date": ["2020-01-08", "2020-01-09", "2020-01-10"],
        "sessions": [10231.0, 12309.0, 12104.0],
    }
)

from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly

from greykite.common.data_loader import DataLoader
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

# Loads dataset into pandas DataFrame
dl = DataLoader()
df = dl.load_peyton_manning()

df = df.rename(columns={"ts": "ds", "sessions": "y"})

# specify dataset information
metadata = MetadataParam(
    time_col="ds",  # name of the time column ("date" in example above)
    value_col="y",  # name of the value column ("sessions" in example above)
    freq="D",  # "H" for hourly, "D" for daily, "W" for weekly, etc.
    # Any format accepted by `pandas.date_range`
)

from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam

model_components = ModelComponentsParam(
    seasonality={
        "yearly_seasonality": False,
        "weekly_seasonality": 2,
    },
    events={"holiday_lookup_countries": []},
    custom={
        # CORRECTED: Use fit_algorithm_dict instead of fit_algorithm
        "fit_algorithm_dict": {"fit_algorithm": "ridge"}  # <-- Fix here
    },
    growth={"growth_term": "linear"},
)


from greykite.framework.templates.autogen.forecast_config import ForecastConfig

config = ForecastConfig(
    model_template="SILVERKITE_EMPTY",
    forecast_horizon=1,  # Reduced from 365 to 1 for 3-day data
    coverage=0.95,
    metadata_param=MetadataParam(time_col="ds", value_col="y", freq="D"),
    model_components_param=model_components,
)

forecaster = Forecaster()
result = forecaster.run_forecast_config(df=df, config=config)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 588, in _fit
    X, fitted_

In [34]:
import pandas as pd
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.autogen.forecast_config import (
    ForecastConfig,
    MetadataParam,
    ModelComponentsParam,
)

# 1. Use proper sample data with sufficient length
df = pd.DataFrame(
    {
        "ds": pd.date_range(start="2020-01-01", periods=14, freq="D"),
        "y": [100, 120, 130, 125, 140, 150, 145, 160, 165, 170, 175, 180, 185, 190],
    }
)

# 2. Simplified model components
model_components = ModelComponentsParam(
    custom={
        "fit_algorithm_dict": {"fit_algorithm": "ridge"},
        "feature_sets_enabled": "auto",  # Required for SILVERKITE_EMPTY
    },
    growth={"growth_term": None},  # Disable trend for short series
    seasonality={"weekly_seasonality": False},  # Disable for < 2 weeks data
    events={"holiday_lookup_countries": []},
)

# 3. Configure with error debugging
config = ForecastConfig(
    model_template="SILVERKITE_EMPTY",
    forecast_horizon=3,
    coverage=0.95,
    metadata_param=MetadataParam(time_col="ds", value_col="y", freq="D"),
    model_components_param=model_components,
)

# 4. Run with detailed error reporting
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=df, config=config
)

Fitting 1 folds for each of 1 candidates, totalling 1 fits


ValueError: 
All the 1 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "/opt/miniconda3/envs/timeseries310/lib/python3.10/site-packages/sklearn/pipeline.py", line 588, in _fit
    X, fitted_